In [ ]:
import keras
from keras.models import Sequential
from keras import layers
import keras.backend as K
import tensorflow as tf
import numpy as np
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import random


In [ ]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
print(gpus)


In [ ]:
df = pd.read_csv('data/accelerometer.csv', delimiter=',')
df.drop(columns=['x', 'y'], inplace=True)
df

In [ ]:
plt.figure(figsize=(20, 6))
plt.plot(df['z'])
plt.xlabel('Index')
plt.ylabel('z')
plt.title('Plot of z values')
plt.show()


In [ ]:
df_dataset = pd.DataFrame()

row_size = 20
increment = 1

for i in range(0, len(df)-row_size, increment):
    df_dataset = pd.concat([df_dataset, df[i:i+row_size].reset_index(drop=True).T])

df_dataset = df_dataset.reset_index(drop=True)
df_dataset

In [ ]:
def add_diff_data(df):
    df_diff = pd.DataFrame()

    for _, row in df.iterrows():
        col_to_col_max_diff = np.float64(sys.float_info.min)

        general_max_diff = row.max() - row.min()

        count = 0
        for col in row:
            if str(col).isdigit():
                count += 1

        for col in range(1, count-1, 1):
            abs_diff = abs(row[col] - row[col-1])
            col_to_col_max_diff = np.max([col_to_col_max_diff, abs_diff])

        df_diff = pd.concat([df_diff, pd.DataFrame({'col_to_col_max_diff': [col_to_col_max_diff], 'general_max_diff': [general_max_diff]})])

    df_diff.reset_index(drop=True, inplace=True)

    df['col_to_col_max_diff'] = df_diff['col_to_col_max_diff']
    df['general_max_diff'] = df_diff['general_max_diff']

add_diff_data(df_dataset)
df_dataset


In [ ]:
plt.figure(figsize=(20, 6))
plt.plot(df['z'], label='z')
plt.plot(df_dataset['col_to_col_max_diff'], label='col_to_col_max_diff')
#plt.plot(df_dataset['general_max_diff'], label='general_max_diff')
plt.xlabel('Index')
plt.ylabel('Value')
plt.title('Plot of z, col_to_col_max_diff, and general_max_diff')
plt.legend()
plt.show()


In [ ]:
min = df_dataset['col_to_col_max_diff'].min()
max = df_dataset['col_to_col_max_diff'].max()
parts = 100

bins = np.linspace(min, max, parts)
labels = range(1, parts)

# Create the histogram
plt.figure(figsize=(10, 6))
plt.hist(df_dataset['col_to_col_max_diff'], bins=bins, edgecolor='black')
plt.xlabel('col_to_col_max_diff')
plt.ylabel('Frequency')
plt.title('Distribution of col_to_col_max_diff')
plt.show()


In [ ]:
def mark_data(df):
    def assign_mark(row):
        if row['col_to_col_max_diff'] < 3000:
            return 'good'
        elif row['col_to_col_max_diff'] < 6000:
            return 'small pits'
        else:
            return 'large pits'

    def assign_nn_answer(row):
        if row['road_state'] == 'good':
            return 0
        elif row['road_state'] == 'small pits':
            return 1
        else:
            return 2

    df['road_state'] = df.apply(assign_mark, axis=1)
    df['nn_answer'] = df.apply(assign_nn_answer, axis=1)


mark_data(df_dataset)
df_dataset


In [ ]:
import os

os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

num_classes = len(df_dataset['road_state'].unique())

print(row_size, num_classes)

model = Sequential([
    layers.Input(shape=(row_size,)),
    layers.Dense(4, activation='tanh'),
    layers.Dense(num_classes, activation='softmax'),
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy', 'categorical_accuracy'])


model.summary()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

X_data = df_dataset.iloc[:, :row_size].values.astype(float)
Y_data = np.array(df_dataset['nn_answer'].tolist()).reshape(-1, 1).astype(float)

print(X_data[0])
print(np.unique(Y_data))

train_input, test_input, train_output, test_output = train_test_split(X_data, Y_data, test_size=0.2, random_state=19191, shuffle=True)

print(train_input.shape)
print(train_output.shape)
print(test_input.shape)
print(test_output.shape)


In [ ]:
history = model.fit(train_input, train_output, epochs=50, batch_size=30, validation_data=(test_input, test_output))


In [ ]:
model.save('../edge/model/model.h5')

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()

In [ ]:
def predict(input):
    results = model.predict(input)
    res = results[0]
    #res = np.where(res == np.max(res), 1, 0)
    return res


user_input = np.array([16516, 16546, 16535, 16533, 16516, 16516, 16549, 16516, 16537, 16544, 16533, 16541, 16551, 16545, 16552, 16525, 16546, 16533, 16753, 16565])
user_input = user_input.reshape(1, -1)
print(predict(user_input))

user_input_shifted = np.array([16546, 16535, 16533, 16516, 16549, 16549, 16549, 16537, 16544, 16533, 16541, 16551, 16545, 16552, 16525, 16546, 16533, 16753, 16565, 16516])
user_input_shifted += 64000
user_input_shifted = user_input_shifted.reshape(1, -1)
print(predict(user_input_shifted))